In [1]:
# 3.3: Grouping, Aggregation and Pivot Tables
import pandas as pd

In [2]:
# 3.3.1: Grouping and Aggregation

In [3]:
# 3.3.1.1: Grouping
df = pd.DataFrame({'gender': ['m', 'm', 'f'], 'rate': [5000, 5500, 6000], 'claims': [4500, 5000, 5500]})
print(df)
df_grouped = df.groupby(['gender']) # can group by multiple columns in list
print(df_grouped)
print(type(df_grouped))

   claims gender  rate
0    4500      m  5000
1    5000      m  5500
2    5500      f  6000
<class 'pandas.core.groupby.DataFrameGroupBy'>


In [4]:
# 3.3.1.2: Standard Aggregation - use if possible due to performance
print('Mean:')
print(df_grouped.mean()) # applies the function to every column in the data frame
print('Sum:')
print(df_grouped['claims'].sum()) # can run on only a subset of columns
print('Count:')
print(df_grouped.count())

Mean:
        claims  rate
gender              
f         5500  6000
m         4750  5250
Sum:
gender
f    5500
m    9500
Name: claims, dtype: int64
Count:
        claims  rate
gender              
f            1     1
m            2     2


In [5]:
# 3.3.1.3: Custom Aggregation
charges = pd.DataFrame({'patient': ['1234', '1234', '1235', '1235', '1235'],
                        'hospital': ['memorial', 'memorial', 'community', 'community', 'community'],
                       'charge': [100, 200, 500, 10, 600],
                      'rev_code': ['0300-lab', '0350-CT', '0610-MRI','0300-lab', '0450-ER']})    

In [6]:
#find the revenue code with the most charges for each patient
charges_grouped = charges.groupby('patient') # use the levels argument with multiple indices
print(charges_grouped.groups) # .groups property of a groupby object returns a dictionary 
                              # with the unique groups as keys and group indices as values

{'1234': Int64Index([0, 1], dtype='int64'), '1235': Int64Index([2, 3, 4], dtype='int64')}


In [7]:
charges_grouped.aggregate({'charge': sum})  # can use shortcut agg function

,charge
patient,
1234,300
1235,1110


In [8]:
def largest_rev_code(x): # assume that x is a sub-data frame
    max_index = x.charge.idxmax() # idxmax function returns index of maximum value
    return x.loc[max_index, 'rev_code'] # the value of the row at the index with maximum value is returned

In [9]:
for group, group_df in charges_grouped: # loop style of iteration through groups
    print(largest_rev_code(group_df))

0350-CT
0450-ER


In [10]:
charges_grouped.aggregate(largest_rev_code) # functional style of iteration through groups

,charge,hospital,rev_code
patient,,,
1234,0350-CT,0350-CT,0350-CT
1235,0450-ER,0450-ER,0450-ER


In [11]:
# 3.3.2.1: Pivot Tables
# index and columns are for dimensions, values is for measures and aggfunc is the aggregation function applied to the measure
pvt = charges.pivot_table(index=['patient'], columns=['hospital'], values=['charge'], aggfunc=[sum, len])
pvt

sum                len         
            charge             charge         
hospital community memorial community memorial
patient                                       
1234           NaN    300.0       NaN      2.0
1235        1110.0      NaN       3.0      NaN

In [12]:
pvt.fillna(0) # manage the NaN's by filling as $0

sum                len         
            charge             charge         
hospital community memorial community memorial
patient                                       
1234           0.0    300.0       0.0      2.0
1235        1110.0      0.0       3.0      0.0